In [1]:
#Dependencies
import pandas as pd
import tweepy
from datetime import datetime as dt
from pprint import pprint
import numpy as np
import re
from time import strftime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from apikeys import consumer_key, consumer_secret, access_token, access_token_secret




import requests
import json
from apikeys import av_key
import pandas as pd
import os
import csv
import sys

In [2]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [37]:
# Target User
target_user = "@realDonaldTrump"


# Variable to hold Tweet Texts, created date, number of retweets, number of likes
tweet_texts = []
created_date =[]
retweets=[]
likes=[]


# Create a loop to iteratively run API requests to bypass weekly tweeter API limit
for x in range(1,160):

    # Get all tweets from Trump home feed (for specified number of pages)
    public_tweets = api.user_timeline(target_user, page=x)

    # Loop through all tweets
    for tweet in public_tweets:
        #Converting text to lower case
        txt = tweet['text'].lower()
        tweet_texts.append(txt)
        created_date.append(tweet["created_at"])
        retweets.append(tweet["retweet_count"])
        likes.append(tweet["favorite_count"])
        
#Created new dictionary to hold above variables
trump_tweets = {
                "Tweet" : tweet_texts,
                "Date" : created_date,
                "Retweets" : retweets,
                "Likes" : likes,
                }

#Converted dictionary to dataframe 
trump_tweets = pd.DataFrame(trump_tweets)
trump_tweets.head(5)

# print(len(trump_tweets))




,Date,Likes,Retweets,Tweet
0,Thu Jun 28 20:49:55 +0000 2018,59895,12850,"prior to departing wisconsin, i was briefed on..."
1,Thu Jun 28 19:09:42 +0000 2018,33943,8109,america is open for business! \nhttps://t.co/f...
2,Thu Jun 28 18:57:22 +0000 2018,60279,14544,"today, we broke ground on a plant that will pr..."
3,Thu Jun 28 13:06:01 +0000 2018,75446,14516,"i am in milwaukee, wisconsin, for meetings. so..."
4,Thu Jun 28 12:56:17 +0000 2018,57398,14001,....persecuted on old and/or totally unrelated...


In [38]:
# Created list of key words
keywords = ['sad','earnings', 'election', 'futures', 'investor', 'low', 'market', 'nasdaq', 'north korea', 'stock',
'market', 'correction', 'trading', 'unemployment','wall street','china','ban',
'bull', 'kim', 'price', 'high', 'corporate', 'global warming', 'tariff', 'tax cut', 'jobs', 'economy', 'trade', 'missile', 'nuclear', 'jobs', 'wages',
'maga']

In [41]:
#Filtered tweets based on keywords and defined variables to hold
filtered_tweet_texts = []
filtered_created_date =[]
filtered_retweets=[]
filtered_likes=[]
filtered_id=[]
keys=[]
#Variables to hold sentiment analysis
compound_list = []
pos_list = []
neu_list = []
neg_list = []
# i = 0

# Looping through tweets
for tweet in trump_tweets.iterrows():
    for key in keywords:
        #Used regular expression to find specific tweets based on keyword
        if re.search(key,tweet[1]['Tweet']):
            #Not all tweets will have keyword, catching tweets that match
            try:
                filtered_tweet_texts.append(tweet[1]["Tweet"])
                filtered_created_date.append(tweet[1]["Date"])
                filtered_retweets.append(tweet[1]["Retweets"])
                filtered_likes.append(tweet[1]["Likes"])
                keys.append(key)
                # Performed sentiment analysis using Vader
                compound = analyzer.polarity_scores(tweet[1]["Tweet"])["compound"]
                pos = analyzer.polarity_scores(tweet[1]["Tweet"])["pos"]
                neu = analyzer.polarity_scores(tweet[1]["Tweet"])["neu"]
                neg = analyzer.polarity_scores(tweet[1]["Tweet"])["neg"]
                compound_list.append(compound)
                pos_list.append(pos)
                neu_list.append(neu)
                neg_list.append(neg)
#                 i += 1   
            except:
                    print('Missing Data')                      
#Create new dictionary for filtered tweets

filtered_trump_tweets = {
                "Tweet" : filtered_tweet_texts,
                "Date" : filtered_created_date,
                "Retweets" : filtered_retweets,
                "Likes" : filtered_likes,
                "Keyword" : keys,
                "Positive" : pos_list,
                "Negative" : neg_list,
                "Neutral" : neu_list,
                "Compound" : compound_list,
                }               

#Convert dictionary to dataframe
filtered_trump_tweets = pd.DataFrame(filtered_trump_tweets)

#Convert date YYYY/MM/DD format
filtered_trump_tweets["Date"] = pd.to_datetime(filtered_trump_tweets["Date"])
filtered_trump_tweets["Date"] = filtered_trump_tweets["Date"].dt.strftime("%Y-%m-%d")

# filtered_trump_tweets.to_csv("DJ_filter.csv", index=False, header=True,encoding='utf-8-sig')


In [42]:
# Grouping by Tweets and concatenating keywords
grouped_keywords = filtered_trump_tweets.groupby(['Tweet'])['Keyword'].apply(list).reset_index()


#merged grouped data frame with filtered data frame
merged_table = pd.merge(grouped_keywords, filtered_trump_tweets, on="Tweet")
# merged_table=merged_table.drop_duplicates(['Tweet'], keep=False, inplace=False)

#Dropped unwanted column
merged_table = merged_table.drop(['Keyword_y'], axis=1)

#Dropped duplicate tweets
merged_table=merged_table.drop_duplicates(['Tweet'], keep='first', inplace=False).reset_index()

#Display
merged_table

,index,Tweet,Keyword_x,Compound,Date,Likes,Negative,Neutral,Positive,Retweets
0,0,"""20,000📈21,000📈22,000📈 23,000📈this year...four...",[maga],0.0000,2017-10-18,55745,0.000,1.000,0.000,13635
1,1,"""after 14 years, u.s. beef hits chinese market...","[market, market, trade]",0.7184,2017-07-12,66419,0.000,0.700,0.300,16740
2,4,"""america's labor market continues to boom"" job...","[market, market, jobs, jobs]",0.0000,2017-07-25,49015,0.000,1.000,0.000,11795
3,8,"""consumer comfort reaches 16-year high on u.s....",[high],0.7351,2017-08-11,50143,0.000,0.556,0.444,11298
4,9,"""consumer confidence hits highest level since ...",[high],0.5106,2017-10-31,39766,0.000,0.769,0.231,9887
5,10,"""consumer confidence soars to highest level si...",[high],0.5106,2017-10-13,46719,0.000,0.732,0.268,10777
6,11,"""corporations have never made as much money as...","[jobs, jobs]",0.4199,2017-08-01,75998,0.000,0.883,0.117,16035
7,13,"""dow passes 23,000 for the first time, fueled ...",[earnings],0.5106,2017-10-17,44359,0.000,0.784,0.216,10862
8,14,"""iran hides behind its assertion of technical ...",[nuclear],-0.6124,2017-10-18,40727,0.227,0.773,0.000,11233
9,15,"""the real big story that affects everybody in ...",[tax cut],0.3818,2018-06-21,47122,0.081,0.775,0.143,11624


In [43]:
# Grouped by date 
grouped_date = merged_table.groupby(['Date']).agg({"Likes": "sum","Retweets": "sum","Compound":"mean",
                                                      "Negative":"mean","Neutral":"mean","Positive":"mean"}).apply(list).reset_index()
grouped_date

,Date,Likes,Retweets,Compound,Negative,Neutral,Positive
0,2017-05-25,0,9612,0.902000,0.000000,0.554000,0.446000
1,2017-05-26,68994,13001,0.784100,0.000000,0.777000,0.223000
2,2017-05-27,171787,37745,0.398933,0.000000,0.821333,0.178667
3,2017-05-29,134427,26799,0.470500,0.084500,0.738000,0.177500
4,2017-05-30,241119,60843,-0.494000,0.241667,0.655000,0.103667
5,2017-05-31,53463,13159,-0.057200,0.050000,0.950000,0.000000
6,2017-06-02,0,20161,0.091067,0.000000,0.965333,0.034667
7,2017-06-03,169653,50250,0.338200,0.139000,0.656000,0.205000
8,2017-06-05,467913,115176,-0.011217,0.133500,0.741000,0.125333
9,2017-06-06,184035,42046,-0.646000,0.276000,0.689500,0.034500


In [44]:
#keeping only extremes of sentiments
# was playing around with the index we will need to figure this out

extreme_dates1 = grouped_date[(grouped_date['Compound'] >= .5) | (grouped_date['Compound'] <= -.5)].reset_index()
# extreme_dates.drop(columns=[1])
print(extreme_dates1.head())
# extremes1 = (grouped_date['Compound'] >= 0.5) ~ (grouped_date['Compound'] <= 0.5)
#print(len(extreme_dates1))

   index        Date   Likes  Retweets  Compound  Negative  Neutral  Positive
0      0  2017-05-25       0      9612    0.9020     0.000   0.5540    0.4460
1      1  2017-05-26   68994     13001    0.7841     0.000   0.7770    0.2230
2      9  2017-06-06  184035     42046   -0.6460     0.276   0.6895    0.0345
3     11  2017-06-09   87654     20608    0.8588     0.000   0.6560    0.3440
4     17  2017-06-17   99793     19862    0.8847     0.000   0.6000    0.4000


In [45]:
extreme_dates1["Date"][0]


'2017-05-25'

In [8]:
###########

In [30]:
base_url = 'https://www.alphavantage.co/query?function='


sector_url = base_url +'SECTOR&apikey='+ av_key


sector_response = requests.get(sector_url)

response_json = sector_response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))


{
    "Meta Data": {
        "Information": "US Sector Performance (realtime & historical)",
        "Last Refreshed": "04:20 PM ET 06/28/2018"
    },
    "Rank A: Real-Time Performance": {
        "Consumer Discretionary": "0.76%",
        "Consumer Staples": "0.17%",
        "Energy": "-0.12%",
        "Financials": "0.86%",
        "Health Care": "0.17%",
        "Industrials": "0.16%",
        "Information Technology": "1.08%",
        "Materials": "0.44%",
        "Real Estate": "0.86%",
        "Telecommunication Services": "2.29%",
        "Utilities": "-0.02%"
    },
    "Rank B: 1 Day Performance": {
        "Consumer Discretionary": "0.76%",
        "Consumer Staples": "0.17%",
        "Energy": "-0.12%",
        "Financials": "0.86%",
        "Health Care": "0.17%",
        "Industrials": "0.16%",
        "Information Technology": "1.08%",
        "Materials": "0.44%",
        "Real Estate": "0.86%",
        "Telecommunication Services": "2.29%",
        "Utilities": "-0.02%

In [46]:
symbol = '^GSPC' # ^GSPC is SP 500


time_series_url = base_url +'TIME_SERIES_DAILY&symbol='+symbol+'&outputsize=full&apikey='+ av_key

daily_response = requests.get(time_series_url)
                          
response_json2 = daily_response.json()   

opening = []
closing = []
date = []




    
    
    


    
for key in response_json2["Time Series (Daily)"]:
    
    opening.append(response_json2["Time Series (Daily)"][key]["1. open"])
    closing.append(response_json2["Time Series (Daily)"][key]["4. close"])
    date.append(key)

sp_df = pd.DataFrame({
    'Date': date,
    'S&P500 Open': opening,
    'S&P500 Close': closing
})


sp_df = sp_df[["Date", "S&P500 Open","S&P500 Close"]]
sp_df = sp_df.sort_values('Date', ascending = True)  

sp_df.head()

,Date,S&P500 Open,S&P500 Close
4651,2000-01-03,1469.2500,1455.2200
4650,2000-01-04,1455.2200,1399.4200
4649,2000-01-05,1399.4200,1402.1100
4648,2000-01-06,1402.1100,1403.4500
4647,2000-01-07,1403.4500,1441.4700


In [31]:
# Loop to calculate

same_day =[]
one_day = []
two_day =[]

for row in range(len(sp_df)):
   same_day_open= float(sp_df["S&P500 Open"][row])
   same_day_close= float(sp_df["S&P500 Close"][row])
   same_day_change = round(((same_day_close - same_day_open)/same_day_open ) * 100 ,2)
   same_day.append(same_day_change)
   
   if row + 1 < len(sp_df):
       prev_day_close= float(sp_df["S&P500 Close"][row])
       next_day_close = float(sp_df["S&P500 Close"][(row + 1)])
       one_day_change = round(((next_day_close - prev_day_close)/prev_day_close ) * 100 ,2)
       one_day.append(one_day_change)
       
   else:
       one_day_change = 0
       one_day.append(one_day_change)

   if row + 2 < len(sp_df):
       prev_day_close= float(sp_df["S&P500 Close"][row])
       
       two_day_close = float(sp_df["S&P500 Close"][(row + 2)])
       two_day_change = round(((two_day_close - prev_day_close)/prev_day_close ) * 100 ,2)
       two_day.append(two_day_change)
       
   else:
       two_day_change = 0
       two_day.append(two_day_change)

   
   
print (same_day[0])
print (same_day[252])

print (one_day[0])
print (one_day[251])

print (two_day[0])
print (two_day[252])

0.65
0.49
-0.61
0.87
0.25
-0.07


In [47]:
sp_df["Same Day Change"] = same_day
sp_df["One Day Change"] = one_day
sp_df["Two Day Change"] = two_day


sp_df.head()

,Date,S&P500 Open,S&P500 Close,Same Day Change,One Day Change,Two Day Change
4651,2000-01-03,1469.2500,1455.2200,0.65,-0.61,0.25
4650,2000-01-04,1455.2200,1399.4200,-1.06,0.87,0.65
4649,2000-01-05,1399.4200,1402.1100,0.03,-0.22,1.17
4648,2000-01-06,1402.1100,1403.4500,-0.94,1.39,1.20
4647,2000-01-07,1403.4500,1441.4700,-0.21,-0.19,0.45


Empty DataFrame
Columns: [index, Created, Likes, Retweets, Compound, Negative, Neutral, Positive]
Index: []
0


In [55]:
merge_table = pd.merge(extreme_dates1, sp_df, on= "Date", how = "left")
merge_table

,index,Date,Likes,Retweets,Compound,Negative,Neutral,Positive,S&P500 Open,S&P500 Close,Same Day Change,One Day Change,Two Day Change
0,0,2017-05-25,0,9612,0.902000,0.000000,0.554000,0.446000,2409.5400,2415.0701,0.36,-0.36,1.41
1,1,2017-05-26,68994,13001,0.784100,0.000000,0.777000,0.223000,2414.5000,2415.8201,-1.75,1.78,1.23
2,9,2017-06-06,184035,42046,-0.646000,0.276000,0.689500,0.034500,2431.9199,2429.3301,-0.50,0.50,0.21
3,11,2017-06-09,87654,20608,0.858800,0.000000,0.656000,0.344000,2436.3899,2431.7700,0.03,-0.03,0.38
4,17,2017-06-17,99793,19862,0.884700,0.000000,0.600000,0.400000,NaN,NaN,NaN,NaN,NaN
5,18,2017-06-18,280464,58869,0.502300,0.090667,0.715333,0.194000,NaN,NaN,NaN,NaN,NaN
6,19,2017-06-19,80330,21636,-0.670500,0.273000,0.596000,0.132000,2442.5500,2453.4600,1.03,-1.02,-1.96
7,21,2017-06-21,380929,84928,0.585550,0.022500,0.710750,0.266750,2439.3101,2435.6101,0.38,-0.38,-0.19
8,23,2017-06-23,51876,9643,0.817600,0.000000,0.667000,0.333000,2434.6499,2438.3000,-2.62,2.69,3.79
9,25,2017-06-25,101128,25257,-0.522000,0.211000,0.695000,0.094000,NaN,NaN,NaN,NaN,NaN


In [54]:
# count = merge_table['S&P500 Open'].value_counts()
# len(count)
merge_table1=merge_table.dropna(axis=0)
print(len(merge_table1))

49


In [57]:
# f_date = dt(2014,7,27)
# l_date = dt(2014, 7, 29)

# delta = l_date - f_date


# print (delta.days)





2
